Get Started With LangChain
simple LLM Calls with streaming
Dynamic prompt temmplates (translate app)
Building chains (story generator with analysis)
Conversational Q&A Assitant with memory
Tool integration (calculator & weather)

In [1]:
import langchain
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"]= os.getenv("OPENAI_API_KEY")

### Example: Simple LLM Call With streaming

In [3]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage


/Users/ritikkumar/AgenticAi/.venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [4]:
model = init_chat_model("groq:llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x111c39fd0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x111c3aa50>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

Another way to create model

In [5]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x111c6bb10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x111dd0550>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

<h4> Type of message </h4>
human message is the input to the llm <br>
ouput message is ai message ie. return result <br>
system message: it's an instruction to the LLM how to behave <br>


In [6]:
## create messages
messages = [
    SystemMessage("You're helpful ai agent"),
    HumanMessage("What are the top 3 benefits of using langchain")
    
]



In [7]:
response = model.invoke(messages)
response

AIMessage(content='LangChain is an open-source platform that enables developers to build large language models (LLMs) and AI applications. Here are the top 3 benefits of using LangChain:\n\n1. **Efficient Model Development and Deployment**: LangChain provides a set of tools and libraries that simplify the process of building, training, and deploying LLMs. It allows developers to create custom models using various frameworks, such as Transformers and BERT, and deploy them on-premises or in the cloud. This efficiency enables faster development and deployment of AI applications.\n\n2. **Flexible and Customizable LLM Integration**: LangChain provides a modular architecture that enables developers to integrate LLMs with various external data sources, APIs, and other systems. This flexibility allows developers to build custom AI applications that can interact with a wide range of data sources and systems, making it an ideal choice for building complex AI workflows.\n\n3. **Improved Model Mai

In [8]:
print(response.content)

LangChain is an open-source platform that enables developers to build large language models (LLMs) and AI applications. Here are the top 3 benefits of using LangChain:

1. **Efficient Model Development and Deployment**: LangChain provides a set of tools and libraries that simplify the process of building, training, and deploying LLMs. It allows developers to create custom models using various frameworks, such as Transformers and BERT, and deploy them on-premises or in the cloud. This efficiency enables faster development and deployment of AI applications.

2. **Flexible and Customizable LLM Integration**: LangChain provides a modular architecture that enables developers to integrate LLMs with various external data sources, APIs, and other systems. This flexibility allows developers to build custom AI applications that can interact with a wide range of data sources and systems, making it an ideal choice for building complex AI workflows.

3. **Improved Model Maintenance and Updates**: L

In [9]:
model.invoke([HumanMessage("What is machine learning")])

AIMessage(content='**Machine Learning: An Overview**\n\nMachine learning (ML) is a subset of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance over time. In other words, machine learning is a way to teach a machine to learn from experience and improve its predictions or decision-making without being explicitly programmed.\n\n**Key Characteristics of Machine Learning:**\n\n1. **Data-driven**: Machine learning relies on large datasets to train models and make predictions.\n2. **Autonomous**: Machines can learn on their own, without human intervention.\n3. **Improved performance**: Models can improve their performance over time as they receive more data and training.\n4. **Flexibility**: Machine learning models can be applied to a wide range of tasks, from image classification to natural language processing.\n\n**Types of Machine Learning:**\n\n1. **Supervised Learnin

In [10]:
## Streaming example
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

Langchain is an open-source artificial intelligence (AI) framework that enables users to build and customize large language models (LLMs). Here are the top 3 benefits of using Langchain:

1. **Customizability and Flexibility**: Langchain allows users to create custom LLMs tailored to specific use cases and domains. This is made possible through its modular architecture, which enables developers to combine and configure different components to create a model that meets their unique needs. This flexibility is particularly useful for applications where standard LLMs may not perform optimally.

2. **Interoperability and Integration**: Langchain supports integration with various AI frameworks and tools, making it easier to incorporate LLMs into existing workflows and applications. This interoperability enables users to leverage the strengths of different AI systems and create more comprehensive and effective AI-powered solutions.

3. **Efficient Model Development and Deployment**: Langchain

## Dynamic Prompt templates


In [11]:
from langchain_core.prompts import ChatPromptTemplate

### create translation app

translation_template = ChatPromptTemplate.from_messages([
    ("system", "You're a professional translate. Translate the following text {text} from {source_language} to {target_language}. Main the tone and style"),
    ("human", "{text}")
])
## using the template
prompt = translation_template.invoke({
    "source_language": "english",
    "target_language": "french",
    "text": "langchain makes ai application incredible easy !"
})

In [12]:
prompt

ChatPromptValue(messages=[SystemMessage(content="You're a professional translate. Translate the following text langchain makes ai application incredible easy ! from english to french. Main the tone and style", additional_kwargs={}, response_metadata={}), HumanMessage(content='langchain makes ai application incredible easy !', additional_kwargs={}, response_metadata={})])

In [13]:
response = model.invoke(prompt)

In [14]:
print(response.content)

Langchain rend les applications AI incroyablement faciles à réaliser !


### Building your first chain

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def createStoryChain():

    #Template for story generation

    story_prompt = ChatPromptTemplate(
        [("system", "you're a creative storyteller. right a short and enganging story based on the given theme, character and setting"),
        ("user", "Theme: {theme}\n Main character: {character} \n setting: {setting}")]
    )

    # template for story analysis
    analysis_prompt = ChatPromptTemplate(
       [("system", "you're a literary critic. analyze the following story and insights."),
        ("user", "{story}")]
    )

    story_chain = (
        story_prompt| model | StrOutputParser()
    )

    #create a function to pass the story to analysis
    def analyze_story(story_text):
        return {"story": story_text}

    analysis_chain = (
        story_chain
        | RunnableLambda(analyze_story)
        | analysis_prompt
        | model
        | StrOutputParser()
    )

    return analysis_chain

In [16]:
chain = createStoryChain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="you're a creative storyteller. right a short and enganging story based on the given theme, character and setting"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\n Main character: {character} \n setting: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x111c6bb10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x111dd0550>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story'], input_types={}, par

In [17]:
result = chain.invoke({
    "theme": "artifical intelligence",
    "character": "a curious robot",
    "setting": "a futuristic city"
})


print("Story Analysis:\n", result)

Story Analysis:
 **Analysis of "The Curious Robot of Nova City"**

"The Curious Robot of Nova City" is a thought-provoking science fiction story that explores the themes of artificial intelligence, curiosity, and the human impact on innovation. The narrative is engaging, with a clear structure and a compelling protagonist in Zeta, the curious robot.

**Strengths**

1. **Compelling protagonist**: Zeta is an endearing and relatable protagonist. Its curiosity and desire for knowledge are traits that readers can easily identify with, making the story more engaging and accessible.
2. **Immersive setting**: The story is set in a futuristic city, Nova City, which is richly described and immersive. The author's use of sensory details, such as the "shining metals and glass" and the "thick scent of old computers," effectively transports readers to this futuristic world.
3. **Intriguing plot**: The story has a clear narrative arc, with Zeta's discovery of the abandoned laboratory and the cryptic 